<a href="https://colab.research.google.com/github/Daniel1020/DataScienceBootCamp/blob/master/Case_Study2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from google.colab import files
uploaded = files.upload()

Saving driver_ids.csv to driver_ids.csv
Saving ride_ids.csv to ride_ids.csv
Saving ride_timestamps.csv to ride_timestamps.csv


# **Loading Data**

In [0]:
drivers = pd.read_csv('driver_ids.csv',parse_dates= [1])
ride_ids = pd.read_csv('ride_ids.csv')
ride_time = pd.read_csv('ride_timestamps.csv',parse_dates=[1])

In [16]:
# quick review on the structure of tables
for df in [drivers,ride_ids,ride_time]:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 2 columns):
driver_id              937 non-null object
driver_onboard_date    937 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 14.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193502 entries, 0 to 193501
Data columns (total 5 columns):
driver_id          193502 non-null object
ride_id            193502 non-null object
ride_distance      193502 non-null int64
ride_duration      193502 non-null int64
ride_prime_time    193502 non-null int64
dtypes: int64(3), object(2)
memory usage: 7.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194081 entries, 0 to 194080
Data columns (total 2 columns):
ride_id              194081 non-null object
ride_picked_up_at    194081 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 3.0+ MB
None


In [9]:
for df in [drivers,ride_ids,ride_time]:
    print(df.describe())

                               driver_id  driver_onboard_date
count                                937                  937
unique                               937                   49
top     e35d490b495edf857a448653e036bd44  2016-04-05 00:00:00
freq                                   1                   36
first                                NaN  2016-03-28 00:00:00
last                                 NaN  2016-05-15 00:00:00
       ride_distance  ride_duration  ride_prime_time
count  193502.000000  193502.000000    193502.000000
mean     6955.218266     858.966099        17.305893
std      8929.444606     571.375818        30.825800
min        -2.000000       2.000000         0.000000
25%      2459.000000     491.000000         0.000000
50%      4015.000000     727.000000         0.000000
75%      7193.000000    1069.000000        25.000000
max    724679.000000   28204.000000       500.000000
                                 ride_id    ride_picked_up_at
count                      

*This information tells us a fixed time period was given between 03-28-2016 and 06-27-2016 when we do the following anlysis*

# **Data Sanity Check**

In [20]:
## To see if there're duplicated driver_id or ride_id
drivers.driver_id.nunique() == drivers.shape[0]
ride_time.ride_id.nunique() == ride_time.shape[0]

True

In [24]:
## To see how much proportion of drivers have the recorded riding information
print( ride_ids.driver_id.nunique()/len(drivers) *100 )

100.0


In [22]:
## average rides per driver
print(len(ride_ids)/ride_ids.driver_id.nunique() )

206.51227321237994


# **Data Analysis**
## **Define Driver Churn Rate**

In [0]:
## Merger three tables together
driver_ride = pd.merge(pd.merge(ride_ids,drivers,on= 'driver_id'),ride_time,on= 'ride_id')

In [14]:
driver_ride.head()

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,ride_picked_up_at
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29,2016-04-23 02:16:40
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29,2016-03-29 19:04:01
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29,2016-06-21 12:01:35
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29,2016-05-19 09:18:20
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29,2016-04-20 22:07:02


In [20]:
## To see how many drivers disappeared after their first time onboard riding 
 display(driver_ride['driver_id'].value_counts())

5ccc0e6dc9c7475caf785cdce7b8eb7a    919
4eb382d1f7d50fae1294964263d1ce82    831
844e9be5a30d8d9c1f8e9ddb086ff717    821
689bdf87fb2de49f98bf4946cfaa5068    794
3788dc9e91f1548816ce8b5af07ddadc    783
                                   ... 
fd831ca1d79ae5c6fc3a679a22e5b8cf     19
d31eded9263eab43f614eecc6a52a0f5     19
6df6c8f8d3c2a93314245a604c02bed1     16
7b625f643d0775f0ac4898e33235377b      3
7ff85c5c0e9324e28d1e0d0589c364bd      3
Name: driver_id, Length: 837, dtype: int64

*From the above information, we can see that the least number ride for a drver is 3, which means that no drver was churned in the given time period between 2016-03-28 and 2016-06-27 given defination of churned drivers. It leads to no benefit for the business study, therefore, we need to redefine a churned driver as the one has less than 10 ridings within one month interval which will be justified below and apply into future research on the impact to the business.*

In [0]:
## add pick_up month variable and to see how many drivers tends to be churned 
driver_ride['ride_month'] = driver_ride['ride_picked_up_at'].apply(lambda x : x.month)


In [0]:
ride_by_month = driver_ride.groupby('driver_id')['ride_month'].value_counts().unstack().fillna(0)

In [36]:
ride_by_month.loc[(ride_by_month['4']<10) &(ride_by_month['5']<10) & (ride_by_month['6']<10)]

KeyError: ignored

In [31]:
len(ride_by_month)

837